In [16]:
import tensorflow.keras as keras

In [17]:
input_dims, hidden_layers, latent_dims = 784, [128, 64], 32

In [22]:
# this is the size of our encoded representations
encoding_dim = latent_dims  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = keras.Input(shape=(input_dims,))
# "encoded" is the encoded representation of the input
encoded = keras.layers.Dense(hidden_layers[0], activation='relu')(input_img)
encoded = keras.layers.Dense(hidden_layers[1], activation='relu')(encoded)
latent = keras.layers.Dense(latent_dims, activation='relu')(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = keras.layers.Dense(hidden_layers[1], activation='relu')(encoded)
decoded = keras.layers.Dense(hidden_layers[0], activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = keras.models.Model(input_img, decoded)

In [28]:
# this model maps an input to its encoded representation
encoder = keras.models.Model(input_img, latent)
encoder.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_14 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_15 (Dense)             (None, 32)                2080      
Total params: 110,816
Trainable params: 110,816
Non-trainable params: 0
_________________________________________________________________


In [30]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = keras.models.Model(encoded_input, decoder_layer(encoded_input))

In [11]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [12]:
from tensorflow.keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

In [14]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [15]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50
235/235 [==============================] - 7s 30ms/step - loss: 0.6959 - val_loss: 0.6958
Epoch 2/50
235/235 [==============================] - 6s 26ms/step - loss: 0.6956 - val_loss: 0.6956
Epoch 3/50
235/235 [==============================] - 6s 26ms/step - loss: 0.6954 - val_loss: 0.6953
Epoch 4/50
235/235 [==============================] - 6s 27ms/step - loss: 0.6951 - val_loss: 0.6951
Epoch 5/50
235/235 [==============================] - 6s 26ms/step - loss: 0.6949 - val_loss: 0.6948
Epoch 6/50
235/235 [==============================] - 7s 28ms/step - loss: 0.6947 - val_loss: 0.6946
Epoch 7/50
235/235 [==============================] - 6s 27ms/step - loss: 0.6944 - val_loss: 0.6944
Epoch 8/50
235/235 [==============================] - 6s 27ms/step - loss: 0.6942 - val_loss: 0.6941
Epoch 9/50
235/235 [==============================] - 7s 29ms/step - loss: 0.6940 - val_loss: 0.6939
Epoch 10/50
235/235 [==============================] - 6s 26ms/step - loss: 0.6938 - val_lo